In [1]:
# Encryption and Decryption Functions
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import os


key = os.urandom(32)  # 256 bits
iv = os.urandom(16)   # 128 bits

def encrypt_message(message):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(message.encode()) + padder.finalize()
    ct = encryptor.update(padded_data) + encryptor.finalize()
    return ct

def decrypt_message(ciphertext):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    unpadder = padding.PKCS7(128).unpadder()
    plaintext = unpadder.update(padded_plaintext) + unpadder.finalize()
    return plaintext.decode()

# Client Implementation
from scapy.all import *
import random
import time

def send_icmp_packet(destination_ip, message):
    encrypted_message = encrypt_message(message)

    packet_size = random.randint(100, 200)
    padding_size = packet_size - len(encrypted_message)
    padding_data = os.urandom(padding_size)
    packet = IP(dst=destination_ip)/ICMP(type=47)/encrypted_message/padding_data
    send(packet)

destination_ip = "192.168.1.1" 

# Continuously prompt for messages and send them
while True:
    message = input("Enter message to send (empty message to exit): ")
    if not message:
        break
    try:
        send_icmp_packet(destination_ip, message)
        print("Message sent successfully.")
    except Exception as e:
        print("Error:", e)

# Server Implementation
def handle_packet(packet):
    try:
        if packet.haslayer(ICMP) and packet[ICMP].type == 47:
            encrypted_payload = bytes(packet[ICMP].payload)
            decrypted_message = decrypt_message(encrypted_payload)
            print("Decrypted message:", decrypted_message)
    except Exception as e:
        print("Error handling packet:", e)

# Start sniffing for ICMP packets
try:
    sniff(filter="icmp", prn=handle_packet)
except KeyboardInterrupt:
    print("Server stopped.")


Error: Permission denied: could not open /dev/bpf0. Make sure to be running Scapy as root ! (sudo)


Scapy_Exception: Permission denied: could not open /dev/bpf0. Make sure to be running Scapy as root ! (sudo)